In [5]:
import pandas as pd

# predictions = pd.read_csv('fn1.7-test-gpt-4o-mini-tempurature-0-xml-descriptions.csv')
# predictions = pd.read_csv('fn1.7-test-gpt-4o-mini-tempurature-1-xml-descriptions-p1.csv')
predictions = pd.read_csv('fn1.7-test-gpt-4o-mini-tempurature-1-xml-descriptions-p2.csv')



In [6]:
import re
def get_tags(text: str) -> dict:
    # example: 'Your gift to Goodwill is important because <Cognizer>people with physical and Mental disabilities</Cognizer> sometimes need an extra hand to <Entity>know</Entity> the pride that comes with work .'
    # output: {'Cognizer': 'people with physical and Mental disabilities', 'Entity': 'know'}
    tags = {}

    # get all text between tags
    matches = re.findall(r'<(.*?)>(.*?)</(.*?)>', text)
    for match in matches:
        tag = match[0]
        content = match[1]
        tags[tag] = content

    return tags


In [8]:
tp = 0
near_miss = 0
fp = 0
fn = 0
tn = 0 # not really used because tags only have positive values, technically all other FEs are TN, but we don't really care about that (maybe we should?)

for label, pred, fes in predictions[['output', 'prediction', 'frame_elements']].values:

    # Get each predicted FE span from the prediction
    pred_tags = get_tags(pred)
    real_tags = eval(fes)

    # Check each predicted FE span
    for tag, content in pred_tags.items():
        if tag.capitalize() in real_tags:
            if content == real_tags[tag.capitalize()]:
                tp += 1
            else:
                fp += 1
                near_miss += 1
        else:
            fp += 1
    
    # Check each real FE span
    for tag, content in real_tags.items():
        if tag not in pred_tags:
            fn += 1
        elif content != pred_tags[tag]:
            fn += 1

# print(f'Perfect: {perfect} out of {len(predictions)}')
print(f'TP: {tp}')
print(f'Missed: {near_miss}')
print(f'FP: {fp}')
print(f'FN: {fn}')

print(f'Precision: {tp / (tp + fp)}')
print(f'Precision (ignoring near misses): {tp / (tp + fp - near_miss)}')
print(f'Recall: {tp / (tp + fn)}')
print(f'F1: {2 * tp / (2 * tp + fp + fn)}')

print(f'Accuracy: {tp / (tp + fp + fn)}')


TP: 3955
Missed: 3621
FP: 8819
FN: 6994
Precision: 0.30961327696884294
Precision (ignoring near misses): 0.43209876543209874
Recall: 0.36122020275824274
F1: 0.33343169076423723
Accuracy: 0.20007082152974504


Bad pipe message: %s [b"A<\\\xc9b\xad\xac\xe4\x0c)Te\x90p\x94\xea\x86'\x00\x01|\x00\x00\x00\x01\x00\x02\x00\x03\x00\x04\x00\x05\x00\x06\x00\x07\x00\x08\x00\t\x00\n\x00\x0b\x00\x0c\x00\r\x00\x0e\x00\x0f\x00\x10\x00\x11\x00\x12\x00\x13\x00\x14\x00\x15\x00\x16\x00\x17\x00\x18\x00\x19\x00\x1a\x00\x1b\x00/\x000\x001\x002\x003\x004\x005\x006\x007\x008\x009\x00:\x00;\x00<\x00=\x00>\x00?\x00@\x00A\x00B\x00C\x00D\x00E\x00F\x00g\x00h\x00i\x00j\x00k\x00l\x00m\x00\x84\x00\x85\x00\x86\x00\x87\x00\x88\x00\x89\x00\x96\x00\x97\x00\x98\x00\x99\x00\x9a\x00\x9b\x00\x9c\x00\x9d\x00\x9e\x00\x9f\x00\xa0\x00\xa1\x00\xa2\x00\xa3\x00\xa4\x00\xa5\x00\xa6\x00\xa7\x00\xba\x00\xbb\x00\xbc\x00\xbd\x00\xbe\x00\xbf\x00\xc0\x00"]
Bad pipe message: %s [b'\xa0\x07\r\x1b_\x0c\x1d\x99\x91\xe6\xc0\xab\x15\xfeE\xd8\xc7\xe5\x00\x01|\x00\x00\x00\x01\x00\x02\x00\x03\x00\x04\x00\x05\x00\x06\x00\x07\x00\x08\x00\t\x00\n\x00\x0b\x00\x0c\x00\r\x00\x0e\x00\x0f\x00\x10\x00\x11\x00\x12\x00\x13\x00\x14\x00\x15\x00\x16\x00\x17\x00\x18\x